<a href="https://colab.research.google.com/github/beltrewilton/cx_analyzer/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Grupo 4  - Proyecto `luzia` 
## Descubrir Tópicos basado en la técnica Latent Dirichlet Allocation [paper](https://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)
### Está aplicado utilizando la base de imdb previamente limpia para ir directo al punto.

In [3]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 1.7 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
from google.colab import files

uploaded = files.upload()

Saving banca_respuestas.csv to banca_respuestas (1).csv


In [10]:
from google.colab import files

uploaded = files.upload()

Saving stopwords_spanish.txt to stopwords_spanish.txt


In [5]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [7]:
stop_spanish = get_stopwords_list("./notebooks/stopwords_spanish.txt")
stop_spanish

stop_spanish.extend(['si','ahi','ahí', 'ah'])

In [8]:
## read the data, separated ith ";"

data = pd.read_csv("./notebooks/dataset/banca_respuestas.csv", sep = ";",
                   encoding_errors="ignore", encoding = 'utf_8')

data_tr = data.copy()

In [9]:
data_tr = data_tr.dropna()
data_tr['razones'] = data_tr['razones'].str.lower()
data_tr['razones'] = data_tr['razones'].replace(',',"")
data_tr['razones'] = data_tr['razones'].replace('.',"")
data_tr['razones'] = data_tr['razones'].replace(';',"")
data_tr

,industria,nps,razones
0,banca,NEUTRAL,tienen buena atencion a la hora de pagar
1,banca,PROMOTOR,es rapido y seguro se puede realizar el pago d...
2,banca,NEUTRAL,a veces el app del bim no funciona para realiz...
3,banca,NEUTRAL,es mas facil y rapido realizar el pago
4,banca,PROMOTOR,es mas rapido y practico desde el celular
...,...,...,...
2466,banca,DETRACTOR,la seorita ultima que yo tengo como a analista...
2467,banca,DETRACTOR,porque es mucho inters y a como yo empec a tra...
2468,banca,DETRACTOR,"porque la verdad yo no recomendara, bueno porq..."
2469,banca,DETRACTOR,"estoy en un grupo de compartamos, estoy en mi ..."


In [10]:
count = CountVectorizer(stop_words= stop_spanish, max_df=0.1, max_features=5000)

In [11]:
X = count.fit_transform(data_tr['razones'].values)

Más información de la implementacion Dirichlet en :

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [35]:
# Esta tarea NLP para descubrir los tópicos consume tiempo, paciencia y café!
n_comp = np.where(len(data. index) < 500, 8,
                  np.where(len(data. index) < 1000, 10,
                           np.where(len(data. index) < 2000, 12,
                                    np.where(len(data. index) < 3000, 15, 20))))

n_comp = n_comp[()]
n_comp = 5
lda = LatentDirichletAllocation(n_components=n_comp, random_state=123, learning_method='batch')


X_topics = lda.fit_transform(X)

A continuación se hace selección de los top 5 tópicos por 10 componentes encontrados.

In [63]:
n_top_words = 1
topics_discovered = []
feature_names = count.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_, 1):
#     print(f'Topic {topic_idx}')
    discovered = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
#     print(' '.join(discovered))
    topics_discovered.append(discovered)
    
# np.array(topics_discovered).reshape(-1, 1)
topics_discovered = [t[0] for t in topics_discovered]
topics_discovered

['crdito', 'ms', 'colas', 'problema', 'sistema']

# **First Stop!!!** Customer must review the topics and assign a name for the identified patterns!

Que sigue?
Mi recomendación es que miremos una implementación aún más robusta de Dirichlet sobre Deep Learning desarrollada por la empresa Uber llamada Pyro, más información:

*   https://medium.com/@anmolmittal_28682/evaluate-probabilistic-topic-models-pyro-latent-dirichlet-allocation-b84b73d110a7
*   https://pyro.ai/examples/intro_long.html




In [64]:
topics = pd.DataFrame(lda.transform(X))
topics_encode = pd.DataFrame(np.where(topics >= 0.3, 1, 0))
other = pd.DataFrame(np.where(topics_encode.sum(axis = 1) == 0, 1, 0)) 
other.columns = ["otros"]

In [65]:
names = []
for topic_idx, topic in enumerate(lda.components_, 1):
    names.append(f'Topic {topic_idx}')

topics_encode.columns = topics_discovered

In [66]:
full_data = data.join(topics_encode) 
full_data = full_data.join(other)

full_data

,industria,nps,razones,crdito,ms,colas,problema,sistema,otros
0,banca,NEUTRAL,TIENEN BUENA ATENCION A LA HORA DE PAGAR,0,0,1,1,0,0
1,banca,PROMOTOR,ES RAPIDO Y SEGURO SE PUEDE REALIZAR EL PAGO D...,0,0,1,0,0,0
2,banca,NEUTRAL,A VECES EL APP DEL BIM NO FUNCIONA PARA REALIZ...,0,0,0,0,1,0
3,banca,NEUTRAL,ES MAS FACIL Y RAPIDO REALIZAR EL PAGO,0,0,0,0,1,0
4,banca,PROMOTOR,ES MAS RAPIDO Y PRACTICO DESDE EL CELULAR,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
2466,banca,DETRACTOR,LA SEORITA ULTIMA QUE YO TENGO COMO A ANALISTA...,0,1,0,0,0,0
2467,banca,DETRACTOR,PORQUE ES MUCHO INTERS Y A COMO YO EMPEC A TRA...,1,1,0,0,0,0
2468,banca,DETRACTOR,"PORQUE LA VERDAD YO NO RECOMENDARA, BUENO PORQ...",1,1,0,0,0,0
2469,banca,DETRACTOR,"ESTOY EN UN GRUPO DE COMPARTAMOS, ESTOY EN MI ...",0,1,0,1,0,0
